### Phase 3: Business & Risk KPIs (1–1.5 Hours) 

#### A. Customer Analytics 

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW customer_count_by_risk_view AS
    SELECT risk_category, COUNT(DISTINCT customer_id) AS customer_count
    FROM silver_customer_account_txn
    GROUP BY risk_category
""")
display(spark.table("customer_count_by_risk_view"))

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW avg_annual_income_by_state_view AS
    SELECT state, AVG(annual_income) AS average_annual_income
    FROM silver_customer_account_txn
    GROUP BY state
""")
display(spark.table("avg_annual_income_by_state_view"))

In [0]:
%sql
select * from silver_customer_account_txn

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW customer_age_distribution_view AS
    SELECT 
        FLOOR(DATEDIFF(current_date(), date_of_birth) / 365.25) AS age,
        COUNT(DISTINCT customer_id) AS customer_count
    FROM silver_customer_account_txn
    GROUP BY age
""")
display(spark.table("customer_age_distribution_view"))

In [0]:
%sql
select * from silver_customer_account_txn

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW top_10_customers_by_txn_amount_view AS
    SELECT customer_id, SUM(amount) AS total_transaction_amount
    FROM silver_customer_account_txn
    GROUP BY customer_id
    ORDER BY total_transaction_amount DESC
    LIMIT 10
""")
display(spark.table("top_10_customers_by_txn_amount_view"))

### B. Account Analytics 

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW active_vs_dormant_accounts_view AS
    SELECT 
        CASE 
            WHEN MAX(DATEDIFF(current_date(), last_transaction_date)) <= 100 THEN 'Active'
            ELSE 'Dormant'
        END AS account_status,
        COUNT(DISTINCT account_id) AS account_count
    FROM silver_customer_account_txn
    GROUP BY account_status
""")
display(spark.table("active_vs_dormant_accounts_view"))

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW avg_balance_by_account_type_view AS
    SELECT account_type, AVG(current_balance) AS average_balance
    FROM silver_customer_account_txn
    GROUP BY account_type
""")
display(spark.table("avg_balance_by_account_type_view"))

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW total_balance_by_branch_view AS
    SELECT branch_code, SUM(current_balance) AS total_balance
    FROM silver_customer_account_txn
    GROUP BY branch_code
""")
display(spark.table("total_balance_by_branch_view"))

In [0]:
MIN_BALANCE_THRESHOLD = 1000  # Set your desired minimum balance threshold

spark.sql(f"""
    CREATE OR REPLACE VIEW accounts_below_min_balance_view AS
    SELECT account_id, current_balance
    FROM silver_customer_account_txn
    WHERE current_balance < {MIN_BALANCE_THRESHOLD}
""")
display(spark.table("accounts_below_min_balance_view"))

### C. Transaction Analytics 

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW monthly_txn_volume_value_view AS
    SELECT 
        YEAR(transaction_date) AS year,
        MONTH(transaction_date) AS month,
        COUNT(*) AS transaction_volume,
        SUM(amount) AS transaction_value
    FROM silver_customer_account_txn
    GROUP BY year, month
    ORDER BY year, month
""")
display(spark.table("monthly_txn_volume_value_view"))

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW failed_vs_completed_txn_ratio_view AS
    SELECT
        status,
        COUNT(*) AS transaction_count
    FROM silver_customer_account_txn
    WHERE status IN ('Failed', 'Completed')
    GROUP BY status
""")
display(spark.table("failed_vs_completed_txn_ratio_view"))

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW most_frequent_transaction_channels_view AS
    SELECT channel, COUNT(*) AS usage_count
    FROM silver_customer_account_txn
    GROUP BY channel
    ORDER BY usage_count DESC
    LIMIT 10
""")
display(spark.table("most_frequent_transaction_channels_view"))

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW merchant_category_contribution_to_revenue_view AS
    SELECT 
        merchant_category,
        SUM(amount) AS total_revenue
    FROM silver_customer_account_txn
    GROUP BY merchant_category
    ORDER BY total_revenue DESC
""")
display(spark.table("merchant_category_contribution_to_revenue_view"))

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW state_wise_transaction_heatmap_view AS
    SELECT 
        state,
        COUNT(*) AS transaction_count,
        SUM(amount) AS total_transaction_value
    FROM silver_customer_account_txn
    GROUP BY state
    ORDER BY transaction_count DESC
""")
df = spark.table("state_wise_transaction_heatmap_view")

import plotly.express as px

pdf = df.toPandas()
fig = px.density_heatmap(
    pdf,
    x="state",
    y="transaction_count",
    z="total_transaction_value",
    color_continuous_scale="Viridis",
    title="State-wise Transaction Heatmap"
)
fig.show()

### D. Risk & Fraud Indicators (Basic) 

In [0]:
%python
HIGH_VALUE_THRESHOLD = 10000  # Set your desired high-value transaction threshold

spark.sql(
    f"""
    CREATE OR REPLACE VIEW high_value_transactions_view AS
    SELECT *
    FROM silver_customer_account_txn
    WHERE cast(amount AS DOUBLE) > {HIGH_VALUE_THRESHOLD}
    """
)
display(spark.table("high_value_transactions_view"))

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW high_risk_failed_transactions_view AS
    SELECT customer_id, account_id, transaction_date, amount, channel
    FROM silver_customer_account_txn
    WHERE risk_category = 'High' AND status = 'Failed'
""")
display(spark.table("high_risk_failed_transactions_view"))

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW dormant_accounts_with_recent_activity_view AS
    SELECT account_id, MAX(last_transaction_date) AS recent_transaction_date
    FROM silver_customer_account_txn
    WHERE DATEDIFF(current_date(), last_transaction_date) > 100
      AND last_transaction_date >= DATE_SUB(current_date(), 30)
    GROUP BY account_id
""")
display(spark.table("dormant_accounts_with_recent_activity_view"))

### Optional KPIs (Stretch Goals) 

In [0]:
%sql
select * from silver_customer_account_txn

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW branch_compliance_score_rankings_view AS
    SELECT 
        branch_code,
        AVG(credit_score) AS avg_compliance_score
    FROM silver_customer_account_txn
    GROUP BY branch_code
    ORDER BY avg_compliance_score DESC
""")
display(spark.table("branch_compliance_score_rankings_view"))

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW customer_churn_indicators_view AS
    SELECT
        customer_id,
        COUNT(DISTINCT account_id) AS num_accounts,
        MAX(last_transaction_date) AS last_transaction_date,
        MIN(current_balance) AS min_balance,
        SUM(CASE WHEN status = 'Failed' THEN 1 ELSE 0 END) AS failed_txn_count,
        SUM(CASE WHEN DATEDIFF(current_date(), last_transaction_date) > 90 THEN 1 ELSE 0 END) AS dormant_account_flag
    FROM silver_customer_account_txn
    GROUP BY customer_id
""")
display(spark.table("customer_churn_indicators_view"))

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW relationship_profitability_metrics_view AS
    SELECT
        customer_id,
        SUM(CAST(amount AS DOUBLE)) AS total_revenue,
        COUNT(DISTINCT account_id) AS num_accounts,
        AVG(current_balance) AS avg_balance,
        SUM(CASE WHEN status = 'Completed' THEN CAST(amount AS DOUBLE) ELSE 0 END) AS completed_txn_value,
        SUM(CASE WHEN status = 'Failed' THEN CAST(amount AS DOUBLE) ELSE 0 END) AS failed_txn_value
    FROM silver_customer_account_txn
    GROUP BY customer_id
""")
display(spark.table("relationship_profitability_metrics_view"))

In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW multi_hop_transaction_network_view AS
    SELECT 
        t1.transaction_id AS txn1_id,
        t1.customer_id AS customer1_id,
        t1.account_id AS account1_id,
        t1.amount AS txn1_amount,
        t1.transaction_date AS txn1_date,
        t2.transaction_id AS txn2_id,
        t2.customer_id AS customer2_id,
        t2.account_id AS account2_id,
        t2.amount AS txn2_amount,
        t2.transaction_date AS txn2_date,
        t3.transaction_id AS txn3_id,
        t3.customer_id AS customer3_id,
        t3.account_id AS account3_id,
        t3.amount AS txn3_amount,
        t3.transaction_date AS txn3_date
    FROM silver_customer_account_txn t1
    INNER JOIN silver_customer_account_txn t2
        ON t1.account_id = t2.account_id AND t1.transaction_id != t2.transaction_id
    INNER JOIN silver_customer_account_txn t3
        ON t2.account_id = t3.account_id AND t2.transaction_id != t3.transaction_id
    WHERE t1.transaction_date < t2.transaction_date AND t2.transaction_date < t3.transaction_date
""")
display(spark.table("multi_hop_transaction_network_view"))